Tensorflow primitives deonstration using Pytorch

In [15]:
import torch
import torch.nn as nn

scalar vector multiplication

In [3]:
scalar = 2
vector = torch.tensor([1, 2, 3])

result = scalar * vector
print(result)


tensor([2, 4, 6])


vector vector multiplications

In [35]:
a = torch.rand(3, 5)
b = torch.rand(5, 2)

result = torch.einsum('ij, jk -> ik', [a, b])
print(result)

tensor([[1.0687, 0.8213],
        [1.3245, 1.1338],
        [1.0876, 0.5421]])


outer product

In [36]:
a = torch.arange(3)
b = torch.arange(3, 7)  
result = torch.einsum('i,j -> ij', [a, b])

print(result)

tensor([[ 0,  0,  0,  0],
        [ 3,  4,  5,  6],
        [ 6,  8, 10, 12]])


scalar dot product

In [37]:
a = torch.arange(6).reshape(3, 2)
b = torch.arange(4).reshape(2, 2)
result = torch.einsum('ij, jk ->', [a, b])

print(result)

tensor(51)


hadamard product

In [38]:
a = torch.arange(6).reshape(2, 3)
b = torch.arange(6,12).reshape(2, 3)
result=torch.einsum('ij,ij->ij', [a, b])
print(result)

tensor([[ 0,  7, 16],
        [27, 40, 55]])


batch matrix multiplication

In [32]:
a = torch.randn(3,2,5)
b = torch.randn(3,5,3)


result=torch.einsum('ijk,ikl->ijl', [a, b])
print(result)

tensor([[[-1.2533,  2.9754,  1.6208],
         [-0.6736, -0.7152,  1.3979]],

        [[ 0.3295,  0.0615, -1.1832],
         [-0.4494, -0.2831,  1.0707]],

        [[ 4.2888,  0.1133, -3.2741],
         [ 2.9413,  0.0150, -0.4242]]])


tensor reduction

In [41]:
a = torch.randn(2, 3, 5, 7)
b = torch.randn(4, 1, 3, 11, 5)
result = torch.einsum('abcd, efghi ->adefh', [a, b])
print(result.shape)
print(result)


torch.Size([2, 7, 4, 1, 11])
tensor([[[[[ 6.0274e+00, -1.0688e+01, -1.5282e+01, -1.9496e+01,  2.7850e+01,
             2.7285e+01, -1.1368e+01, -5.1229e+00,  1.4111e+01,  2.3682e+01,
            -2.5848e+00]],

          [[-3.6405e+00,  7.5461e-01, -1.1874e+01,  9.7234e+00,  1.7538e+01,
            -1.4558e+01,  2.4650e+01, -2.5525e+00, -2.3424e+01,  8.0706e+00,
            -1.8275e+01]],

          [[ 1.2167e+01, -3.5243e+01, -1.0339e+00, -2.5644e+00, -1.1607e+01,
             2.2875e+01, -1.8194e+01, -1.8708e+01,  3.0346e+01, -1.6445e+01,
             1.4835e+01]],

          [[ 1.0912e+01,  3.2397e+00,  2.1487e+01, -2.7279e+01,  1.0307e+00,
             1.0587e+01, -6.6202e+00, -2.5640e+01, -1.2861e+01,  7.6414e+00,
             1.4723e+00]]],


         [[[ 3.5920e+00, -6.3696e+00, -9.1071e+00, -1.1618e+01,  1.6597e+01,
             1.6260e+01, -6.7745e+00, -3.0529e+00,  8.4093e+00,  1.4113e+01,
            -1.5404e+00]],

          [[-2.1695e+00,  4.4970e-01, -7.0764e+00,  5.7946e

transpose

In [31]:
a = torch.arange(6).reshape(2, 3)
result=torch.einsum('ij->ji', [a])
print(result) 

tensor([[0, 3],
        [1, 4],
        [2, 5]])


Bilinear transformation

In [29]:
a = torch.randn(2,3)
b = torch.randn(5,3,7)
c = torch.randn(2,7)
output=torch.einsum('ik,jkl,il->ij', [a, b, c])

print(output)

tensor([[-9.7473, -6.2321,  2.4311, -2.5698, -0.0524],
        [ 2.6706, -1.9656, -1.2043,  2.3952,  3.2716]])


attention

In [28]:
def random_tensors(shape, num=1, requires_grad=False):
  tensors = [torch.randn(shape, requires_grad=requires_grad) for i in range(0, num)]
  return tensors[0] if num == 1 else tensors

# Parameters
# -- [hidden_dimension]
bM, br, w = random_tensors([7], num=3, requires_grad=True)
# [hidden_dimension x hidden_dimension]
WY, Wh, Wr, Wt = random_tensors([7, 7], num=4, requires_grad=True)

def attention(Y, ht, rt1):
  # [batch_size x hidden_dimension] 
  tmp = torch.einsum('ik, kl -> il', [ht, Wh]) + torch.einsum('ik, kl -> il', [rt1, Wr])

  Mt = torch.tanh(torch.einsum('ijk, kl -> ijl', [Y, WY]) + tmp.unsqueeze(1).expand_as(Y) + bM)
  
  # [batch_size x sequence_length]
  at = torch.nn.functional.softmax(torch.einsum('ijk, k -> ij', [Mt, w])) 
  
  # [batch_size x hidden_dimension]
  rt = torch.einsum('ijk, ij -> ik', [Y, at]) + torch.tanh(torch.einsum('ij, jk -> ik', [rt1, Wt]) + br)
  
  return rt, at

# Inputs - [batch_size x sequence_length x hidden_dimension]
Y = torch.randn(3,5,7)
# [batch_size x hidden_dimension]
ht, rt1 = random_tensors([3, 7], num=2)

rt, at = attention(Y, ht, rt1)

print(at)

tensor([[0.0411, 0.1094, 0.0123, 0.7748, 0.0623],
        [0.0760, 0.0673, 0.0664, 0.7233, 0.0670],
        [0.0048, 0.4568, 0.0429, 0.0151, 0.4804]], grad_fn=<SoftmaxBackward0>)


<ipython-input-28-ef75e35febdf>:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  at = torch.nn.functional.softmax(torch.einsum('ijk, k -> ij', [Mt, w]))


tree qn

In [23]:
def transition(zl):
  # -- [batch_size x num_actions x hidden_dimension]
  return zl.unsqueeze(1) + torch.tanh(torch.einsum("bk,aki->bai", [zl, W]) + b)

# Sampled dummy inputs
# -- [batch_size x hidden_dimension]
zl =torch.rand(3,4)
b = torch.randn(4, 4)
W = torch.randn(4, 4, 4)
transition(zl)

tensor([[[ 1.0928, -0.2138, -0.1242, -0.3267],
         [ 1.0904, -0.1676,  1.4390,  0.9047],
         [ 1.0592,  1.3200,  0.4016,  0.5025],
         [ 1.0443,  0.3764,  0.9564, -0.1828]],

        [[ 1.1388, -0.5354, -0.3156,  0.2913],
         [ 1.1432, -0.4621,  1.2699,  0.9428],
         [ 1.0894,  1.2044,  0.4178,  0.1974],
         [ 1.1239, -0.3746,  0.7737,  0.2189]],

        [[ 1.4470, -0.0330, -0.8625, -0.0819],
         [ 1.1244,  0.0516,  1.0501,  0.4459],
         [ 1.3419,  1.9345, -0.8358, -0.1164],
         [ 1.3772,  0.7978,  0.5285, -0.1329]]])